In [1]:
import os
os.chdir('../')

## 1. 将checkpoints中inference的结果存储到ckpt中

In [3]:
# # 删除 ckpt 中的的inference结果清除
# dirs = os.listdir('./ckpt')
# for dir in dirs:
#     if dir == ".DS_Store" or dir == "STEP":
#         continue
#     sub_dirs = os.listdir('./ckpt/' + dir)
#     for sub_dir in sub_dirs:
#         if sub_dir == ".DS_Store":
#             continue
#         files = os.listdir('./ckpt/' + dir + '/' + sub_dir)
#         for file in files:
#             if file == ".DS_Store":
#                 continue
#             if 'validate' in file:
#                 os.system('rm ./ckpt/' + dir + '/' + sub_dir + '/' + file)
#                 print('rm ./ckpt/' + dir + '/' + sub_dir + '/' + file)


In [4]:
# # 将inference结果复制到ckpt对应文件夹下
# dirs = os.listdir('./checkpoints')
# for dir in dirs:
#     sub_dirs = os.listdir('./checkpoints/' + dir)
#     for sub_dir in sub_dirs:
#         files = os.listdir('./checkpoints/' + dir + '/' + sub_dir)
#         inference_results = None
#         model_name = None
#         data_name = None
#         for file in files:
#             if 'validate' in file:
#                 inference_results = file
#             if dir.split("_")[0] in file:
#                 model_name = file.split("_")[0]
#                 data_name = file.split("_")[1][:-3]
#         # copy inference_results to ckpt/model_name/data_name
#         if inference_results:
#             os.system('cp ./checkpoints/' + dir + '/' + sub_dir + '/' + inference_results + ' ./ckpt/' + model_name + '/' + data_name + '/')
#             print('cp ./checkpoints/' + dir + '/' + sub_dir + '/' + inference_results + ' ./ckpt/' + model_name + '/' + data_name + '/')

## 读取结果并产生LaTeX表格

In [2]:
# 给定模型，读取其结果
def read_txt(path):
    with open(path, 'r') as f:
        lines = f.readlines()
    return lines

def read_model_results(model_name, data_name):
    try:
        files = os.listdir('./ckpt/' + model_name + '/' + data_name)
    except FileNotFoundError:
        return [10000, 10000, 10000, 10000, 10000, 10000]
    for file in files:
        if 'train' in file:
            training_results = file
    # 读取training logs，得到average training time
    content = read_txt('./ckpt/' + model_name + '/' + data_name + "/" + training_results)
    # get average trainign time\
    training_time = []
    for line in content:
        if "Result <train>:" in line:
            training_time.append(float(line.split("train_time: ")[1].split(" (s)")[0]))
    AVG_TRAINING_TIME = sum(training_time) / len(training_time)
    # 读取inference logs，得到metrics以及#param
    target_index = 0
    for index, line in enumerate(content):
        if "_best_val_MAE.pt" in line:
            target_index = index

    for line in content[target_index:]:
        if "Result <test>:" in line:
            break

    MAE = float(line.split("test_MAE: ")[1].split(",")[0])
    RMSE = float(line.split("test_RMSE: ")[1].split(",")[0])
    MAPE = float(line.split("test_MAPE: ")[1].split(",")[0]) * 100
    WAPE = float(line.split("test_WAPE: ")[1].split(",")[0]) * 100
    for line in content:
        if "Number of parameters:" in line: break
    PARAM = int(line.split("Number of parameters: ")[1]) / 1000000
    return [MAE, RMSE, MAPE, WAPE, PARAM, AVG_TRAINING_TIME]


In [3]:
def get_latex_str(data_list, model_list, metric_list, type):
    # get data
    all_data = {}
    for data_name in data_list:
        for model_name in model_list:
            # if model_name in ['STEP', 'Triformer', 'Crossformer']: 
            #     [MAE, RMSE, MAPE, WAPE, PARAM, AVG_TRAINING_TIME] = [0, 0, 0, 0, 0, 0]
            # else:
            [MAE, RMSE, MAPE, WAPE, PARAM, AVG_TRAINING_TIME] = read_model_results(model_name, data_name)
            # 全部保留两位小数，并变成str。
            # 注意3.10要保持3.10，而不是3.1
            if data_name == "ExchangeRate":
                MAE = "%.4f" % MAE
                RMSE = "%.4f" % RMSE
                MAPE = "%.4f" % MAPE + "\%"
                WAPE = "%.4f" % WAPE + "\%"
                PARAM = "%.2f" % PARAM
                AVG_TRAINING_TIME = "%.2f" % AVG_TRAINING_TIME
            else:
                MAE = "%.2f" % MAE
                RMSE = "%.2f" % RMSE
                MAPE = "%.2f" % MAPE + "\%"
                WAPE = "%.2f" % WAPE + "\%"
                PARAM = "%.2f" % PARAM
                AVG_TRAINING_TIME = "%.2f" % AVG_TRAINING_TIME
            all_data[data_name + '_' + model_name + "_MAE"] = MAE
            all_data[data_name + '_' + model_name + "_RMSE"] = RMSE
            all_data[data_name + '_' + model_name + "_MAPE"] = MAPE
            all_data[data_name + '_' + model_name + "_WAPE"] = WAPE
            all_data[data_name + '_' + model_name + "_Param"] = PARAM
            all_data[data_name + '_' + model_name + "_Speed"] = AVG_TRAINING_TIME
    # print results
    for data_name in data_list:
        for metric_name in metric_list:
            line = [metric_name]
            for model_name in model_list:
                if data_name + '_' + model_name + "_" + metric_name in all_data.keys():
                    line.append(all_data[data_name + '_' + model_name + "_" + metric_name])
                else:
                    line.append('N/A')
                # if model_name in ['STEP', 'Triformer', 'Crossformer']: 
                #     line.append(' ')
                # else:
                #     line.append(all_data[data_name + '_' + model_name + "_" + metric_name])
            if metric_name == "Param":
                if type == "STF":
                    print(" \cmidrule{2-14}")
                else:
                    print(" \cmidrule{2-12}")
            line = get_bold_latex(line=line, data_name=data_name)
            a = 1
            line = " & " + " & ".join([str(x) for x in line]) + r" \\"
            if metric_name == "MAE":
                if type == "STF":
                    line = r"\multirow{6}*{\rotatebox{90}{\textbf{" + data_name + "}}} " + line
                else:
                    line = r"\multirow{5}*{\rotatebox{90}{\textbf{" + data_name + "}}} " + line
                line = r"\midrule" + "\n" + line
            print(line)

def get_bold_latex(line, data_name):
    metric_name = line[0]
    line = line[1:]
    if len(line) == 12:
        type = "STF"
    else:
        type = "LTSF"
    new_line = []
    for num in line:
        if num == "N/A":
            new_line.append(1e10)
        else:
            if metric_name in ["MAPE", "WAPE"]:
                new_line.append(float(num[:-2]))
            else:
                new_line.append(float(num))
    if type == "STF":
        cate_1 = new_line[:5]
        cate_2 = new_line[5:10]
        cate_3 = new_line[10:]
        # get min index (maybe multiple)
        min_index_1 = get_min_indexes(cate_1)
        min_index_2 = get_min_indexes(cate_2)
        min_index_3 = get_min_indexes(cate_3)
        min_index = get_min_indexes(new_line)
        for index in min_index_1:
            new_line[index] = r"\underline{\textbf{" + "%.2f" % new_line[index] + "}}"
        for index in min_index_2:
            new_line[index + 5] = r"\underline{\textbf{" + "%.2f" % new_line[index + 5] + "}}"
        for index in min_index_3:
            new_line[index + 10] = r"\underline{\textbf{" + "%.2f" % new_line[index + 10] + "}}"
        for index in min_index:
            new_line[index] = r"\color{teal}{" + new_line[index] + "}" 
    else:
        cate_1 = new_line[:7]
        cate_2 = new_line[7:]
        # get min index (maybe multiple)
        min_index_1 = get_min_indexes(cate_1)
        min_index_2 = get_min_indexes(cate_2)
        min_index = get_min_indexes(new_line)
        for index in min_index_1:
            if data_name == "ExchangeRate":
                new_line[index] = r"\underline{\textbf{" + "%.4f" % new_line[index] + "}}"
            else:
                new_line[index] = r"\underline{\textbf{" + "%.2f" % new_line[index] + "}}"
        for inde in min_index_2:
            if data_name == "ExchangeRate":
                new_line[inde + 7] = r"\underline{\textbf{" + "%.4f" % new_line[inde + 7] + "}}"
            else:
                new_line[inde + 7] = r"\underline{\textbf{" + "%.2f" % new_line[inde + 7] + "}}"
        for index in min_index:
            new_line[index] = r"\color{teal}{" + str(new_line[index]) + "}" 
    # convert float to str (保留两位小数)
    result = []
    for _ in new_line:
        try:
            if data_name == "ExchangeRate":
                result.append("%.4f" % float(_))
            else:
                result.append("%.2f" % float(_))
        except:
            result.append(_)
    if metric_name in ["MAPE", "WAPE"]:
        for index, _ in enumerate(result):
            for i in range(len(_)-1, -1, -1):
                if _[i] != "}":
                    result[index] = _[:i+1] + "\%" + _[i+1:]
                    break
    for index, _ in enumerate(result):
        if _ == "10000.00" or _ == "10000.00\\%":
            result[index] = "N/A"
    return [metric_name] + result

def get_min_indexes(cate):
    min_index = []
    min_value = min(cate)
    for index, value in enumerate(cate):
        if value == min_value:
            min_index.append(index)
    return min_index

In [4]:
# 产生LaTeX表格
DATA_LIST = ['METR-LA', 'PEMS-BAY', 'PEMS03', 'PEMS04', 'PEMS07', 'PEMS08']
MODEL_LIST = ['STGCN', 'DCRNN', 'GWNet', 'DGCRN', 'D2STGNN', 'AGCRN', 'MTGNN', 'StemGNN', 'GTS', 'STEP', 'STNorm', 'STID']
METRIC_LIST = ['MAE', 'RMSE', 'MAPE', 'WAPE', "Param", "Speed"]
get_latex_str(DATA_LIST, MODEL_LIST, METRIC_LIST, "STF")


\midrule
\multirow{6}*{\rotatebox{90}{\textbf{METR-LA}}}  & MAE & 3.11 & 3.03 & 3.03 & 2.94 & \color{teal}{\underline{\textbf{2.88}}} & 3.16 & 3.05 & 3.72 & 3.13 & \underline{\textbf{2.93}} & 3.14 & \underline{\textbf{3.12}} \\
 & RMSE & 6.31 & 6.23 & 6.12 & 6.04 & \color{teal}{\underline{\textbf{5.91}}} & 6.44 & 6.16 & 7.33 & 6.32 & \underline{\textbf{5.96}} & \underline{\textbf{6.49}} & \underline{\textbf{6.49}} \\
 & MAPE & 8.37\% & 8.31\% & 8.17\% & \color{teal}{\underline{\textbf{7.79\%}}} & 7.83\% & 8.88\% & 8.16\% & 10.43\% & 8.62\% & \underline{\textbf{8.00\%}} & \underline{\textbf{8.84\%}} & 9.13\% \\
 & WAPE & 5.38\% & 5.24\% & 5.24\% & 5.10\% & \color{teal}{\underline{\textbf{4.99\%}}} & 5.48\% & 5.28\% & 6.45\% & 5.42\% & \underline{\textbf{5.07\%}} & 5.44\% & \underline{\textbf{5.40\%}} \\
 \cmidrule{2-14}
 & Param & 0.25 & 0.37 & 0.31 & \underline{\textbf{0.20}} & 0.39 & 0.75 & \underline{\textbf{0.41}} & 1.20 & 38.49 & 40.48 & 0.22 & \color{teal}{\underline{\textbf{0.12}

In [5]:
# 产生LaTeX表格
DATA_LIST = ["PEMS04", "PEMS08", "ETTh1", "ETTh2", "ETTm1", "ETTm2", "Electricity", "Weather", "ExchangeRate"]
MODEL_LIST = ["Informer", "Autoformer", "Pyraformer", "FEDformer", "Triformer", "Crossformer", "PatchTST", "Linear", "DLinear", "NLinear"]
METRIC_LIST = ['MAE', 'RMSE', 'WAPE', "Param", "Speed"]
get_latex_str(DATA_LIST, MODEL_LIST, METRIC_LIST, "LTSF")


\midrule
\multirow{5}*{\rotatebox{90}{\textbf{PEMS04}}}  & MAE & 27.94 & 40.21 & 25.49 & 26.89 & \color{teal}{\underline{\textbf{23.81}}} & 26.75 & 29.02 & \underline{\textbf{37.42}} & 37.52 & 37.62 \\
 & RMSE & 44.74 & 56.46 & 41.74 & 41.46 & \color{teal}{\underline{\textbf{39.42}}} & 45.24 & 48.39 & \underline{\textbf{62.14}} & 62.21 & 62.38 \\
 & WAPE & 12.84\% & 18.52\% & 11.72\% & 12.39\% & \color{teal}{\underline{\textbf{10.95\%}}} & 12.31\% & 13.35\% & \underline{\textbf{17.22\%}} & 17.26\% & 17.31\% \\
 \cmidrule{2-12}
 & Param & 12.40 & 12.07 & 218.42 & 18.36 & \underline{\textbf{1.69}} & 13.51 & 2.21 & \color{teal}{\underline{\textbf{0.11}}} & 0.23 & \color{teal}{\underline{\textbf{0.11}}} \\
 & Speed & 28.62 & 137.61 & \underline{\textbf{25.22}} & 147.75 & 313.52 & 338.94 & 164.01 & \color{teal}{\underline{\textbf{21.13}}} & 21.24 & 21.27 \\
\midrule
\multirow{5}*{\rotatebox{90}{\textbf{PEMS08}}}  & MAE & 26.92 & 35.56 & 22.03 & 25.14 & \color{teal}{\underline{\textbf{18.74}